# 情報システム実習論　テキスト分析課題2A
### 必要なライブラリ、データセットのインポート

In [12]:
!pip install pandas
!pip install nltk
!pip install gensim

  Using cached gensim-3.8.3-cp37-cp37m-macosx_10_9_x86_64.whl (24.2 MB)
     |████████████████████████████████| 116 kB 2.2 MB/s 
  Using cached boto-2.49.0-py2.py3-none-any.whl (1.4 MB)
     |████████████████████████████████| 128 kB 3.7 MB/s 
  Using cached jmespath-0.10.0-py2.py3-none-any.whl (24 kB)
     |████████████████████████████████| 6.3 MB 1.8 MB/s 
  Using cached s3transfer-0.3.3-py2.py3-none-any.whl (69 kB)
  Using cached docutils-0.15.2-py3-none-any.whl (547 kB)
  Created wheel for smart-open: filename=smart_open-2.1.0-py3-none-any.whl size=110317 sha256=0b20baa33777c0be20378d1be3fcfdad6c194dadc19e3ab8509171732adab6a3
  Stored in directory: /Users/kento/Library/Caches/pip/wheels/56/b5/6d/86dbe4f29d4688e5163a8b8c6b740494310040286fca4dc648
Successfully built smart-open


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import collections
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

### nltkが使えるようにしておく

In [14]:
nltk.download("stopwords")
nltk.download("wordnet"z)
nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /Users/kento/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kento/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package reuters to /Users/kento/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kento/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### データを取得

In [10]:
from nltk.corpus import reuters as corpuszzzzzzz

### データセットの中身確認

In [19]:
print(corpus.words(fileids) for field in corpus.fileids()[1])

<generator object <genexpr> at 0x7fbf65cd2dd0>


### 全document数

In [16]:
len(corpus.fileids())

10788

###  前から1000文とってきて解析を行う

In [28]:
k=1000
docs = [corpus.words(fileid) for fileid in corpus.fileids()[:k]]
print(docs[1])
print(docs[2])
tags = corpus.fileids()[:k]
print(tags)

['CHINA', 'DAILY', 'SAYS', 'VERMIN', 'EAT', '7', '-', ...]
['JAPAN', 'TO', 'REVISE', 'LONG', '-', 'TERM', ...]
['test/14826', 'test/14828', 'test/14829', 'test/14832', 'test/14833', 'test/14839', 'test/14840', 'test/14841', 'test/14842', 'test/14843', 'test/14844', 'test/14849', 'test/14852', 'test/14854', 'test/14858', 'test/14859', 'test/14860', 'test/14861', 'test/14862', 'test/14863', 'test/14865', 'test/14867', 'test/14872', 'test/14873', 'test/14875', 'test/14876', 'test/14877', 'test/14881', 'test/14882', 'test/14885', 'test/14886', 'test/14888', 'test/14890', 'test/14891', 'test/14892', 'test/14899', 'test/14900', 'test/14903', 'test/14904', 'test/14907', 'test/14909', 'test/14911', 'test/14912', 'test/14913', 'test/14918', 'test/14919', 'test/14921', 'test/14922', 'test/14923', 'test/14926', 'test/14928', 'test/14930', 'test/14931', 'test/14932', 'test/14933', 'test/14934', 'test/14941', 'test/14943', 'test/14949', 'test/14951', 'test/14954', 'test/14957', 'test/14958', 'test/

## 前処理を行う
### nltkストップワードの作成(数字や記号も消す)

In [30]:
en_stop = nltk.corpus.stopwords.words('english')
en_stop= ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<"]                  \
         +["0","1","2","3","4","5","6","7","8","9","10","11","12","86","1986","1987","000"]                                                      \
         +["said","say","u","v","mln","ct","net","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","price"]          \
         +en_stop

### 前処理関数の作成

In [31]:
from nltk.corpus import wordnet as wn #lemmatize関数のためのimport

def preprocess_word(word, stopwordset):
    
    #1.make words lower ex: Python =>python
    word=word.lower()
    
    #2.remove "," and "."
    if word in [",","."]:
        return None
    
    #3.remove stopword  ex: the => (None) 
    if word in stopwordset:
        return None
    
    #4.lemmatize  ex: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    elif lemma in stopwordset: #lemmatizeしたものがstopwordである可能性がある
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

### 前処理の結果を出力
### 前処理前

In [36]:
print(docs[0][:25])

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears']


### 前処理後

In [35]:
print(preprocess_documents(docs)[0][:25])

['asian', 'exporter', 'fear', 'damage', 'japan', 'rift', 'mounting', 'trade', 'friction', 'japan', 'raise', 'fear', 'among', 'many', 'asia', 'exporting', 'nation', 'row', 'could', 'inflict', 'far', 'reaching', 'economic', 'damage', 'businessmen']


 ## クラスタリングパート
 ### tf-idfで前処理を施した文章をベクトル化
 ### vectorizerを使用する。(ハイパーパラメータを設定する)

In [39]:
pre_docs=preprocess_documents(docs)
pre_docs=["".join(doc) for doc in pre_docs]
print(pre_docs[0])

vectorizer = TfidfVectorizer(max_features=200, token_pattern=u'(?u)\\b\\w+\\b' )

asianexporterfeardamagejapanriftmountingtradefrictionjapanraisefearamongmanyasiaexportingnationrowcouldinflictfarreachingeconomicdamagebusinessmenofficialtellreutercorrespondentasiancapitalmovejapanmightboostprotectionistsentimentleadcurbamericanimportproductexporterconflictwouldhurtlongrunshorttermtokyolossmightgainimpose300tariffimportjapaneseelectronicsgoodapril17retaliationjapanallegefailurestickpactsellsemiconductorworldmarketcostunofficialjapaneseestimateputimpacttariffspokesmanmajorelectronicsfirmwouldvirtuallyhaltexportproducthitnewtaxablebusinessspokesmanleadingjapaneseelectronicsfirmmatsushitaelectricindustrialcoltdmc>.tariffremainplacelengthtimebeyondmonthmeancompleteerosionexportgoodsubjecttarifftommurthastockanalysttokyoofficebrokerjamescapelco>.taiwanbusinessmenofficialalsoworryawareseriousnessthreatjapanservewarningusseniortaiwanesetradeofficialasknametaiwantradetradesurplus15last95surplushelpswelltaiwanforeignexchangereserves53amongworldlargemustquicklyopenmarketremovet

### fitさせる

In [41]:
tf_idf = vectorizer.fit_transform(pre_docs)